In [22]:
import pandas as pd
from des_helpers import dpa_select_function, encrypt, decrypt, bin2hex, des, preprocess_key, dec2bin

# Load data
trace_df = pd.read_csv("processed_trace_data.csv")

# Constants
kc = 15  # Known correct key index
ciphertexts = trace_df['Ciphertext']

In [23]:
trace_df.iloc[0]

Filepath      secmatv1_2006_04_0809/wave_DES_HW_2006-04-09_0...
Key                                            6a65786a65786a65
Plaintext                                      c80e9f96b23813ff
Ciphertext                                     8d28b211cf8d2683
Power                                                  0.072367
Name: 0, dtype: object

In [24]:
# Test encrypt decrypt
sample = trace_df.iloc[0]
key = sample["Key"].upper()
pt = sample["Plaintext"].upper()
ct = sample["Ciphertext"].upper()

text = bin2hex(encrypt(pt, key))
print(text == ct)

text = bin2hex(decrypt(ct, key))
print(text == pt)

True
True


In [25]:
# Test select func
def mod_decrypt(ciphertext, key):
	rkb = preprocess_key(key)
	rkb_rev = rkb[::-1]
	text, sbox_out = des(ciphertext, rkb_rev)

	return text, sbox_out

text, sbox_out = mod_decrypt(ct, key)
dec2bin(sbox_out[0][0]) # round 0 box 0

'0110'

In [27]:
# Test on entire dataset
results = []
for ct in [c.upper() for c in ciphertexts]:
    text, sbox_out = mod_decrypt(ct, key)
    decrypt_out = int(dec2bin(sbox_out[0][0])[0])
    select_out = dpa_select_function(ct, 60) # correct key is 60 from the paper
    
    if decrypt_out == select_out: 
        results.append(True)
    else:
        print(decrypt_out, select_out)
        results.append(False)
        
print(all(results))

True
